In [ ]:
#!pip install slack-sdk webdriver_manager PyKakao pycountry ccy


In [ ]:
from openvpn_api import VPN
from Misc import *
from selenium.webdriver.common.by import By
import re
from selenium.common.exceptions import NoSuchElementException


In [ ]:
driver = getMyWebDriver("CHROME",isHeadless=False)


In [ ]:
URL="https://www.youtube.com/premium"

In [ ]:
def open_youtube_url():
    driver.implicitly_wait(0)
    driver.get(URL)

    #EUR
    try:
        driver.find_element(By.XPATH,'/html/body/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/form[1]/div/div/button').click()
    except Exception as e:
        pass
    open_family_url=[
    '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/yt-unlimited-page-header-renderer/div[2]/yt-collapsible-ypc-offer-renderer/yt-formatted-string/a',
    '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/yt-section-group-view-model/yt-lp-section-group-view-model/div/div[2]/undefined/div/div[5]/span/span[2]/a',
    ]
    driver.implicitly_wait(0)

    for myurl in open_family_url:
        found=False
        print(myurl)
        try:
            test1 = driver.find_element(By.XPATH,myurl)
            found = True
        except:
            pass
        if found == True:
            try:
                test1.click()
            except:
                pass
            break


In [ ]:
def read_api_keys(filename):
    api_keys = {}
    with open(filename, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            api_keys[key] = value
    return api_keys

naverkey=read_api_keys("naver_key.txt")

def papago_translate(text, source_lang='en', target_lang='ko', client_id=naverkey['ID'], client_secret=naverkey['KEY']):
    # Papago API 엔드포인트 URL
    url = 'https://openapi.naver.com/v1/papago/n2mt'

    # 요청 헤더 설정
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }

    # 요청 데이터 설정
    data = {
        'source': source_lang,
        'target': target_lang,
        'text': text
    }

    try:
        # 번역 요청 보내기
        response = requests.post(url, headers=headers, data=data)
        
        # HTTP 상태코드 확인
        if response.status_code == 200:
            result = response.json()
            translated_text = result['message']['result']['translatedText']
            return translated_text
        else:
            return f"번역 실패. HTTP 오류 코드: {response.status_code}"

    except Exception as e:
        return f"번역 실패: {str(e)}"

In [ ]:
mykey=read_api_keys("naver_key.txt")
exchange_rate=dict()

In [ ]:

def get_currency_convert(fromCode,toCode,price,mykey=mykey['EXCHANGE_KEY']):
    global exchange_rate
    if fromCode not in exchange_rate or toCode not in exchange_rate:
        print("read exchange")
        url = f"https://openexchangerates.org/api/latest.json?app_id={mykey}"
        headers = {"accept": "application/json"}
        response = requests.get(url, headers=headers)
        if(response.status_code !=200):
            raise BaseException()
        else:
            exchange_rate=response.json()['rates']
    from_to_usd = float(price)/float(exchange_rate[fromCode])
    usd_to_to = float(from_to_usd)*float(exchange_rate[toCode])
    # print(exchange_rate)
    return usd_to_to


In [ ]:
import pycountry
import ccy
def get_country_code(driver):
    if 'is not available in your country' in driver.page_source:
        return None
    country_code = driver.find_element(By.XPATH,'/html/body/ytd-app/div[1]/div/ytd-masthead/div[4]/div[1]/ytd-topbar-logo-renderer/span').text
    if country_code=="":
        country_code="US"
    
    country = pycountry.countries.get(alpha_2=country_code)
    eng_name = country.name
    currency = ccy.countryccy(country_code)
    if(currency == None):
        currency = "USD"
    
    data = {'CODE':country_code,
            'ENGNAME':eng_name,
            'CURRENCY':currency,
            'KO_NAME':papago_translate(eng_name),
            'JA_NAME':papago_translate(eng_name,target_lang='ja'),
            'CN_NAME':papago_translate(eng_name,target_lang='zh-CN')}
    return data

In [ ]:
ovpn_dir = "./openvpn_config/"
ovpn_files = [f for f in os.listdir(ovpn_dir) if f.endswith('.ovpn')]

In [ ]:
import locale
locale.setlocale(locale.LC_ALL,'ko_KR.UTF-8')

pattern = r'((TRY|ARS|THB|CHF|SEK|SGD|RSD|RON|NOK|CZK|UAH|ZAR|HUF|AED|DKK|[$€£¥₩₹])\s?\d+(?:[.,]\d+)?)'
def get_price_elements(driver,fromCode):
    prices_t = re.findall(pattern,driver.page_source)
    prices = [match[0] for match in prices_t]
    print(prices)
    temp_prices = sorted(set(prices),key=lambda x: float(re.sub(r'[^0-9.]','',x)))
    test_prices=['0','0','0','0']
    #student, individual, family, annual
    if len(temp_prices)==1:
        test_prices[1]=temp_prices[0]
    else:
        test_prices[:len(temp_prices)]=temp_prices[:len(temp_prices)]

    # print(test_prices)
    usd_prices =  [f"${get_currency_convert(fromCode,'USD',float(re.sub(r'[^0-9.]','',x))):.2f}"  for x in test_prices]
    krw_prices =  [f"₩{locale.format_string('%d', int(get_currency_convert(fromCode,'KRW',float(re.sub(r'[^0-9.]','',x))))                             ,grouping=True)}" for x in  test_prices]
    return [test_prices,usd_prices,krw_prices]


In [ ]:
import subprocess
import psutil
import csv

ovpnkey=read_api_keys("naver_key.txt")
with open('ovpn_id_pw.txt','w') as file:
    file.write(ovpnkey['OVPN_ID'])
    file.write('\n')
    file.write(ovpnkey['OVPN_PW'])
    file.close()

def append_to_csv(filename,data):
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)

for ovpn_file in ovpn_files:
    ovpn_path = os.path.join(ovpn_dir,ovpn_file)

    with open(ovpn_path,'r') as file:
        ovpn_contents = file.read()
    mystring = f'auth-user-pass {os.getcwd()}/ovpn_id_pw.txt'
    ovpn_contents = ovpn_contents.replace('auth-user-pass',mystring)
    temp_ovpn_file = 'temp.ovpn'
    with open(temp_ovpn_file,'w') as file:
        file.write(ovpn_contents)
    
    openvpn_command=f"sudo openvpn --config {temp_ovpn_file}"

    try:
        vpn = subprocess.Popen(openvpn_command, shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        print(f'Error: {e}')
    time.sleep(10)
    open_youtube_url()


    parent = psutil.Process(vpn.pid)
    for child in parent.children(recursive=True):
        subprocess.Popen(f'sudo kill -9 {child.pid}',shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    vpn.terminate()
    vpn.wait()


    print("DONE")
    os.remove(temp_ovpn_file)
    country_dict = get_country_code(driver)

    if country_dict!=None:
        if country_dict['CODE'].lower()==ovpn_file[0:2].lower():
            price_dict = get_price_elements(driver,country_dict['CURRENCY'])
            print(country_dict)
            print(price_dict)
            row = [ovpn_file]+list(country_dict.values())+[element for sublist in price_dict for element in sublist]
        else:
            row = [ovpn_file,"NA"]
        append_to_csv('gogo.csv',row)
    else:
        row = [ovpn_file]
        append_to_csv('gogo.csv',row)

                                                                                                            
                                                                                                                
        

In [ ]:
os.exit(0)

In [ ]:

open_youtube_url()
country_dict = get_country_code(driver)
if country_dict!=None:
    print(country_dict['CURRENCY'])
    price_dict = get_price_elements(driver,country_dict['CURRENCY'])
    print(ovpn_file)
    print(country_dict)
    print(price_dict)
    row = [ovpn_file]+list(country_dict.values())+[element for sublist in price_dict for element in sublist]
    print(row)
    append_to_csv('gogo.csv',row)
else:
    print("Failed:"+ovpn_file)




## END

In [ ]:
#END

{
  "AED": "United Arab Emirates Dirham",
  "AFN": "Afghan Afghani",
  "ALL": "Albanian Lek",
  "AMD": "Armenian Dram",
  "ANG": "Netherlands Antillean Guilder",
  "AOA": "Angolan Kwanza",
  "ARS": "Argentine Peso",
  "AUD": "Australian Dollar",
  "AWG": "Aruban Florin",
  "AZN": "Azerbaijani Manat",
  "BAM": "Bosnia-Herzegovina Convertible Mark",
  "BBD": "Barbadian Dollar",
  "BDT": "Bangladeshi Taka",
  "BGN": "Bulgarian Lev",
  "BHD": "Bahraini Dinar",
  "BIF": "Burundian Franc",
  "BMD": "Bermudan Dollar",
  "BND": "Brunei Dollar",
  "BOB": "Bolivian Boliviano",
  "BRL": "Brazilian Real",
  "BSD": "Bahamian Dollar",
  "BTC": "Bitcoin",
  "BTN": "Bhutanese Ngultrum",
  "BWP": "Botswanan Pula",
  "BYN": "Belarusian Ruble",
  "BZD": "Belize Dollar",
  "CAD": "Canadian Dollar",
  "CDF": "Congolese Franc",
  "CHF": "Swiss Franc",
  "CLF": "Chilean Unit of Account (UF)",
  "CLP": "Chilean Peso",
  "CNH": "Chinese Yuan (Offshore)",
  "CNY": "Chinese Yuan",
  "COP": "Colombian Peso",
  "CRC": "Costa Rican Colón",
  "CUC": "Cuban Convertible Peso",
  "CUP": "Cuban Peso",
  "CVE": "Cape Verdean Escudo",
  "CZK": "Czech Republic Koruna",
  "DJF": "Djiboutian Franc",
  "DKK": "Danish Krone",
  "DOP": "Dominican Peso",
  "DZD": "Algerian Dinar",
  "EGP": "Egyptian Pound",
  "ERN": "Eritrean Nakfa",
  "ETB": "Ethiopian Birr",
  "EUR": "Euro",
  "FJD": "Fijian Dollar",
  "FKP": "Falkland Islands Pound",
  "GBP": "British Pound Sterling",
  "GEL": "Georgian Lari",
  "GGP": "Guernsey Pound",
  "GHS": "Ghanaian Cedi",
  "GIP": "Gibraltar Pound",
  "GMD": "Gambian Dalasi",
  "GNF": "Guinean Franc",
  "GTQ": "Guatemalan Quetzal",
  "GYD": "Guyanaese Dollar",
  "HKD": "Hong Kong Dollar",
  "HNL": "Honduran Lempira",
  "HRK": "Croatian Kuna",
  "HTG": "Haitian Gourde",
  "HUF": "Hungarian Forint",
  "IDR": "Indonesian Rupiah",
  "ILS": "Israeli New Sheqel",
  "IMP": "Manx pound",
  "INR": "Indian Rupee",
  "IQD": "Iraqi Dinar",
  "IRR": "Iranian Rial",
  "ISK": "Icelandic Króna",
  "JEP": "Jersey Pound",
  "JMD": "Jamaican Dollar",
  "JOD": "Jordanian Dinar",
  "JPY": "Japanese Yen",
  "KES": "Kenyan Shilling",
  "KGS": "Kyrgystani Som",
  "KHR": "Cambodian Riel",
  "KMF": "Comorian Franc",
  "KPW": "North Korean Won",
  "KRW": "South Korean Won",
  "KWD": "Kuwaiti Dinar",
  "KYD": "Cayman Islands Dollar",
  "KZT": "Kazakhstani Tenge",
  "LAK": "Laotian Kip",
  "LBP": "Lebanese Pound",
  "LKR": "Sri Lankan Rupee",
  "LRD": "Liberian Dollar",
  "LSL": "Lesotho Loti",
  "LYD": "Libyan Dinar",
  "MAD": "Moroccan Dirham",
  "MDL": "Moldovan Leu",
  "MGA": "Malagasy Ariary",
  "MKD": "Macedonian Denar",
  "MMK": "Myanma Kyat",
  "MNT": "Mongolian Tugrik",
  "MOP": "Macanese Pataca",
  "MRU": "Mauritanian Ouguiya",
  "MUR": "Mauritian Rupee",
  "MVR": "Maldivian Rufiyaa",
  "MWK": "Malawian Kwacha",
  "MXN": "Mexican Peso",
  "MYR": "Malaysian Ringgit",
  "MZN": "Mozambican Metical",
  "NAD": "Namibian Dollar",
  "NGN": "Nigerian Naira",
  "NIO": "Nicaraguan Córdoba",
  "NOK": "Norwegian Krone",
  "NPR": "Nepalese Rupee",
  "NZD": "New Zealand Dollar",
  "OMR": "Omani Rial",
  "PAB": "Panamanian Balboa",
  "PEN": "Peruvian Nuevo Sol",
  "PGK": "Papua New Guinean Kina",
  "PHP": "Philippine Peso",
  "PKR": "Pakistani Rupee",
  "PLN": "Polish Zloty",
  "PYG": "Paraguayan Guarani",
  "QAR": "Qatari Rial",
  "RON": "Romanian Leu",
  "RSD": "Serbian Dinar",
  "RUB": "Russian Ruble",
  "RWF": "Rwandan Franc",
  "SAR": "Saudi Riyal",
  "SBD": "Solomon Islands Dollar",
  "SCR": "Seychellois Rupee",
  "SDG": "Sudanese Pound",
  "SEK": "Swedish Krona",
  "SGD": "Singapore Dollar",
  "SHP": "Saint Helena Pound",
  "SLL": "Sierra Leonean Leone",
  "SOS": "Somali Shilling",
  "SRD": "Surinamese Dollar",
  "SSP": "South Sudanese Pound",
  "STD": "São Tomé and Príncipe Dobra (pre-2018)",
  "STN": "São Tomé and Príncipe Dobra",
  "SVC": "Salvadoran Colón",
  "SYP": "Syrian Pound",
  "SZL": "Swazi Lilangeni",
  "THB": "Thai Baht",
  "TJS": "Tajikistani Somoni",
  "TMT": "Turkmenistani Manat",
  "TND": "Tunisian Dinar",
  "TOP": "Tongan Pa'anga",
  "TRY": "Turkish Lira",
  "TTD": "Trinidad and Tobago Dollar",
  "TWD": "New Taiwan Dollar",
  "TZS": "Tanzanian Shilling",
  "UAH": "Ukrainian Hryvnia",
  "UGX": "Ugandan Shilling",
  "USD": "United States Dollar",
  "UYU": "Uruguayan Peso",
  "UZS": "Uzbekistan Som",
  "VEF": "Venezuelan Bolívar Fuerte (Old)",
  "VES": "Venezuelan Bolívar Soberano",
  "VND": "Vietnamese Dong",
  "VUV": "Vanuatu Vatu",
  "WST": "Samoan Tala",
  "XAF": "CFA Franc BEAC",
  "XAG": "Silver Ounce",
  "XAU": "Gold Ounce",
  "XCD": "East Caribbean Dollar",
  "XDR": "Special Drawing Rights",
  "XOF": "CFA Franc BCEAO",
  "XPD": "Palladium Ounce",
  "XPF": "CFP Franc",
  "XPT": "Platinum Ounce",
  "YER": "Yemeni Rial",
  "ZAR": "South African Rand",
  "ZMW": "Zambian Kwacha",
  "ZWL": "Zimbabwean Dollar"
}

In [ ]:
import tempfile
import subprocess
import base64
import time

country='in'
i=6
try:
    vpn_data = requests.get("http://www.vpngate.net/api/iphone/").text.replace("\r", "")
    servers = [line.split(",") for line in vpn_data.split("\n")]
    labels = servers[1]
    labels[0] = labels[0][1:]
    servers = [s for s in servers[2:] if len(s) > 1]
except BaseException:
    print("Cannot get VPN servers data")
    exit(1)

desired = [s for s in servers if country.lower() in s[i].lower()]
found = len(desired)
print("Found " + str(found) + " servers for country " + country)
if found == 0:
    exit(1)

supported = [s for s in desired if len(s[-1]) > 0]
print(str(len(supported)) + " of these servers support OpenVPN")
# We pick the best servers by score
winner = sorted(supported, key=lambda s: float(s[2].replace(",", ".")), reverse=True)[0]

print("\n== Best server ==")
pairs = list(zip(labels, winner))[:-1]
for (l, d) in pairs[:4]:
    print(l + ": " + d)
print(pairs[4][0] + ": " + str(float(pairs[4][1]) / 10 ** 6) + " MBps")
print("Country: " + pairs[5][1])

print("\nLaunching VPN...")
_, path = tempfile.mkstemp()

f = open(path, "w")
f.write(base64.b64decode(winner[-1]).decode())
f.write(
    "\nscript-security 2\nup /etc/openvpn/update-resolv-conf\ndown /etc/openvpn/update-resolv-conf"
)
f.close()
x = subprocess.Popen(["sudo", "openvpn", "--config", path])